In [34]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
import pandas as pd

# Going through the example provided in:
# https://brightdata.com/blog/how-tos/web-scraping-with-python

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

In [35]:
browser = webdriver.Firefox()
url = 'https://cs.deals/fi/market/csgo/?sort=discount&sort_desc=1'
browser.get(url)
browser.implicitly_wait(0.5)
time.sleep(10)
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")
courts_list = soup.find_all("div", class_ = "item csgo")
#print(courts_list)

[<div class="item csgo" style="border-bottom: 2px solid rgb(75, 105, 255);"><div class="top"><div class="name">Danger Close</div><div class="subName">Field-Tested</div><div class="info"><div class="more"><span class="val"></span><i aria-hidden="true" class="far fa-times"></i></div></div><div class="pic"><img alt="StatTrak™ SG 553 | Danger Close (Field-Tested)" class="itemimg" loading="lazy" src="/steamItems/MTQ1Njkw_2.png"/><div class="type stattrak">StatTrak™ SG 553</div></div></div><div class="stickers"></div><img alt="CSGO" class="appIcon" src="/images/steam/appicons/730.png"/><div class="price">0.11 €<span class="discount">-35.29%</span></div><div class="floatBar" style="display: none;"><i aria-hidden="true" class="fas fa-caret-down"></i></div><div class="locktime"><i aria-hidden="true" class="fal fa-lock"></i> 6 päivää</div><div class="addToCartWrapper"><div class="addToCartBtnWrapper"><span class="addToCartBtn">Lisää koriin</span></div><div class="menuBtnWrapper"><i aria-hidden="

In [58]:
# Create a list of 100 first items ordered by discount
# [name, price, discount]
complete_list = []
str_courts_list = str(courts_list)

# Add item discounts to the list
discounts_list = re.findall("\"discount\">(.*?)%</span>", str_courts_list)
discounts_list_rdy = []
for discount in discounts_list:
    if discount != "?":
        i_float = float(discount) # Discounts are needed in float
        discounts_list_rdy.append(i_float)

# Add item prices to the list
prices_list = re.findall("(price\">|small\">)(.*?)€", str_courts_list)
prices_list_rdy = []
for j,prices in prices_list:
    prices_changed = prices.replace(",","")
    prices_changed2 = prices_changed.replace(" ", "")
    prices_list_rdy.append(float(prices_changed2)) # Prices are also needed in float

# Add item names to the list
names_list_rdy = re.findall("<img alt=\"(.*?)\"", str_courts_list)
for name in names_list_rdy:
    if name == "CSGO":
        names_list_rdy.remove(name)
    elif name == "sticker":
        names_list_rdy.remove(name)

# Combine all lists
for row in range(len(discounts_list_rdy)):
    complete_list.append([names_list_rdy[row], prices_list_rdy[row], discounts_list_rdy[row]])

for item in complete_list:
    print(item)


['StatTrak™ SG 553 | Danger Close (Field-Tested)', 0.11, -35.29]
['Revolution Case', 0.28, -33.33]
['Chroma 3 Case', 1.54, -30.0]
['★ Huntsman Knife | Autotronic (Field-Tested)', 183.34, -26.64]
['Kilowatt Case', 1.68, -25.0]
['Desert Eagle | Conspiracy (Factory New)', 8.0, -23.59]
['Kilowatt Case', 1.71, -23.66]
['StatTrak™ M4A4 | Griffin (Field-Tested)', 3.5, -23.41]
['P250 | Verdigris (Minimal Wear)', 0.31, -20.51]
['★ Bayonet | Freehand (Field-Tested)', 276.22, -21.59]
['★ Stiletto Knife | Urban Masked (Well-Worn)', 175.76, -21.52]
['AK-47 | Neon Revolution (Field-Tested)', 13.47, -21.46]
['AWP | Medusa (Minimal Wear)', 2574.15, -20.81]
['AK-47 | Legion of Anubis (Field-Tested)', 6.0, -20.74]
['Glock-18 | Weasel (Battle-Scarred)', 0.28, -20.0]
['AWP | Asiimov (Field-Tested)', 115.75, -20.47]
['StatTrak™ AWP | Asiimov (Battle-Scarred)', 106.97, -20.23]
['★ StatTrak™ Huntsman Knife | Lore (Field-Tested)', 210.0, -20.17]
['AWP | Asiimov (Well-Worn)', 88.21, -19.92]
['AK-47 | Bloodspor

In [60]:
# Categorize the interesting items
interesting_list = []
for item in complete_list:
    if item[1] > 5 and item[2] <= -30: # Interested in high enough price and low enough discount
        interesting_list.append(item)

for i in interesting_list:
    print(i)

